# 基于Tensorflow实现word2vec

## 流程

1. 数据：可以是一篇文章或多篇文章，词与词之间要有逻辑关系。
2. 数据处理：词转换成索引。
3. 数据清洗：按照词频，忽略出现次数少的。
4. batch：输入什么？输出什么？也是这个任务中的重点。
5. model：CBOW，skip-gram，这里选用skip-gram。输入一个词，输出上下文。

## 导入工具包

In [1]:
import collections
import os
import random
import urllib
import zipfile
import datetime
import numpy as np
import tensorflow as tf

## 设置参数

In [2]:
# 训练参数
learning_rate = 0.1
batch_size = 128
num_steps = 3000000 # 训练次数
display_step = 10000 # 多少次显示损失值
eval_step = 200000 # 多少次验证一下

# 测试样例，用来进行验证
eval_words = ['nine', 'of', 'going', 'hardware', 'american', 'britain']

# Word2Vec 参数
embedding_size = 200 # 词向量维度
max_vocabulary_size = 50000 # 语料库大小，不重复的词的个数
min_occurrence = 10 # 最小词频
skip_window = 3 # 左右窗口大小
num_skips = 2 # 一次制作多少个输入输出对
num_sampled = 64 # 负采样

## 加载数据

In [3]:
# 加载训练数据，其实什么数据都行
data_path = 'text8.zip'
with zipfile.ZipFile(data_path) as f:
    text_words = f.read(f.namelist()[0]).lower().split()

In [4]:
# 看一下单词个数
len(text_words)

17005207

## 创建一个计数器

In [5]:
# 计算每个词出现了多少次
count = [('UNK', -1)]
# 基于词频返回max_vocabulary_size个常用词
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size - 1))

In [6]:
# 打印一下，看看前10个
count[0:10]

[('UNK', -1),
 (b'the', 1061396),
 (b'of', 593677),
 (b'and', 416629),
 (b'one', 411764),
 (b'in', 372201),
 (b'a', 325873),
 (b'to', 316376),
 (b'zero', 264975),
 (b'nine', 250430)]

- 默认已经是排序好了的

- 别忘了，咱们还设置了min_occurrence参数，需要判断每一个词是否满足给定条件

## 清除词频过低的单词

In [7]:
# 剔除掉出现次数少于'min_occurrence'的词
for i in range(len(count) - 1, -1, -1): # 从start到end每次step多少
    if count[i][1] < min_occurrence:
        count.pop(i)
    else:
        # 判断时，从小到大排序的，所以跳出时候剩下的都是满足条件的
        break

## 词-ID映射

In [8]:
# 计算语料库大小
vocabulary_size = len(count)
# 每个词都分配一个ID，字典结构
word2id = dict()
for i, (word, _)in enumerate(count):
    word2id[word] = i

In [9]:
word2id

{'UNK': 0,
 b'the': 1,
 b'of': 2,
 b'and': 3,
 b'one': 4,
 b'in': 5,
 b'a': 6,
 b'to': 7,
 b'zero': 8,
 b'nine': 9,
 b'two': 10,
 b'is': 11,
 b'as': 12,
 b'eight': 13,
 b'for': 14,
 b's': 15,
 b'five': 16,
 b'three': 17,
 b'was': 18,
 b'by': 19,
 b'that': 20,
 b'four': 21,
 b'six': 22,
 b'seven': 23,
 b'with': 24,
 b'on': 25,
 b'are': 26,
 b'it': 27,
 b'from': 28,
 b'or': 29,
 b'his': 30,
 b'an': 31,
 b'be': 32,
 b'this': 33,
 b'which': 34,
 b'at': 35,
 b'he': 36,
 b'also': 37,
 b'not': 38,
 b'have': 39,
 b'were': 40,
 b'has': 41,
 b'but': 42,
 b'other': 43,
 b'their': 44,
 b'its': 45,
 b'first': 46,
 b'they': 47,
 b'some': 48,
 b'had': 49,
 b'all': 50,
 b'more': 51,
 b'most': 52,
 b'can': 53,
 b'been': 54,
 b'such': 55,
 b'many': 56,
 b'who': 57,
 b'new': 58,
 b'used': 59,
 b'there': 60,
 b'after': 61,
 b'when': 62,
 b'into': 63,
 b'american': 64,
 b'time': 65,
 b'these': 66,
 b'only': 67,
 b'see': 68,
 b'may': 69,
 b'than': 70,
 b'world': 71,
 b'i': 72,
 b'b': 73,
 b'would': 74,
 b'd

## 所有词转换成ID

In [10]:
data = list()
unk_count = 0
for word in text_words:
    # 全部转换成id
    index = word2id.get(word, 0)
    if index == 0:
        unk_count += 1
    data.append(index)

count[0] = ('UNK', unk_count)
# 还要做一个id2word，用来检验结果
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", vocabulary_size)
print("Most common words:", count[:10])

Words count: 17005207
Unique words: 253854
Vocabulary size: 47135
Most common words: [('UNK', 444176), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]


## 构建所需训练数据

In [11]:
data_index = 0
# 不断生成输入输出对
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    # 输入
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    # 标签
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # 7为窗口大小，左3右3中间1
    span = 2 * skip_window + 1
    # 创建一个长度为7的队列
    buffer = collections.deque(maxlen=span)
    # 循环器，都取完了，再重头来一次。
    if data_index + span > len(data):
        data_index = 0
    # 队列里存的是当前窗口，例如deque([5234, 3081, 12, 6, 195, 2, 3134], maxlen=7)
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    # 遍历
    # num_skips表示取多少组不同的词作为输出，此例为2
    for i in range(batch_size // num_skips):
        # 上下文就是[0, 1, 2, 4, 5, 6]
        context_words = [w for w in range(span) if w != skip_window]
        # 在上下文里随机选2个候选词
        words_to_use = random.sample(context_words, num_skips)
        # 遍历每一个候选词，用其当做输出也就是标签
        for j, context_word in enumerate(words_to_use):
            # 输入都为当前窗口的中间词，即3
            batch[i * num_skips + j] = buffer[skip_window]
            # 用当前候选词当做标签
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            # 之前已经传入7个词了，窗口要右移了，例如原来为[5234, 3081, 12, 6, 195, 2, 3134]，
            # 现在为[3081, 12, 6, 195, 2, 3134, 46]
            buffer.append(data[data_index])
            data_index += 1

    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

## 定义参数

In [12]:
with tf.device('/cpu:0'):
    # 维度：47135, 200
    # embedding，初始向量
    embedding = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
    # 负采样
    # 权重参数
    nce_weights = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
    # 偏置项参数
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

## 通过tf.nn.embedding_lookup函数将索引转换成词向量

In [13]:
def get_embedding(x):
    with tf.device('/cpu:0'):
        x_embed = tf.nn.embedding_lookup(embedding, x)
        return x_embed

## 损失函数定义
- 先分别计算出正样本和采样出的负样本对应的output和label
- 再通过 sigmoid cross entropy来计算output和label的loss

In [14]:
def nce_loss(x_embed, y):
    with tf.device('/cpu:0'):
        y = tf.cast(y, tf.int64)
        loss = tf.reduce_mean(
            tf.nn.nce_loss(weights=nce_weights,
                           biases=nce_biases,
                           labels=y,
                           inputs=x_embed,
                           # 采样出多少个负样本
                           num_sampled=num_sampled,
                           num_classes=vocabulary_size))
        return loss

## 评估模块

In [15]:
# Evaluation.
def evaluate(x_embed):
    with tf.device('/cpu:0'):
        # 单个词的向量
        x_embed = tf.cast(x_embed, tf.float32)
        # 归一化
        x_embed_norm = x_embed / tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
        # 全部向量的
        embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True), tf.float32)
        # 计算余弦，判断相似度
        cosine_sim_op = tf.matmul(x_embed_norm, embedding_norm, transpose_b=True)
        return cosine_sim_op

## 优化及执行优化

In [16]:
# 优化器
optimizer = tf.optimizers.SGD(learning_rate)
# 迭代优化
def run_optimization(x, y):
    with tf.device('/cpu:0'):
        with tf.GradientTape() as g:
            emb = get_embedding(x)
            loss = nce_loss(emb, y)

        # 计算梯度
        gradients = g.gradient(loss, [embedding, nce_weights, nce_biases])

        # 更新
        optimizer.apply_gradients(zip(gradients, [embedding, nce_weights, nce_biases]))

## 训练模型

In [ ]:
# 获取测试单词
x_test = np.array([word2id[w.encode('utf-8')] for w in eval_words])

# 计时开始
time1 = datetime.datetime.now()
# 训练
for step in range(1, num_steps + 1):
    batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
    run_optimization(batch_x, batch_y)
    # 打印结果
    if step % display_step == 0 or step == 1:
        loss = nce_loss(get_embedding(batch_x), batch_y)
        print("step: %i, loss: %f" % (step, loss))
        
    # Evaluation.
    if step % eval_step == 0 or step == 1:
        print("Evaluation...")
        sim = evaluate(get_embedding(x_test)).numpy()
        for i in range(len(eval_words)):
            # 返回前8个最相似的
            top_k = 8
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = '"%s" nearest neighbors:' % eval_words[i]
            for k in range(top_k):
                log_str = '%s %s,' % (log_str, id2word[nearest[k]])
            print(log_str)
            
print(datetime.datetime.now() - time1)

step: 1, loss: 493.687012
Evaluation...
"nine" nearest neighbors: b'expiry', b'ethnic', b'domains', b'croatians', b'slaps', b'twh', b'handicraft', b'containers',
"of" nearest neighbors: b'erupts', b'aging', b'turing', b'grammarian', b'compliant', b'cgpm', b'vitrification', b'values',
"going" nearest neighbors: b'nunn', b'pros', b'sparing', b'copperfield', b'graphical', b'heist', b'wrongs', b'lak',
"hardware" nearest neighbors: b'vinland', b'stitched', b'klingon', b'conspicuously', b'mcghee', b'procedure', b'starred', b'walden',
"american" nearest neighbors: b'studios', b'rahul', b'switches', b'reinforces', b'splinter', b'spokesman', b'quatre', b'roundabout',
"britain" nearest neighbors: b'obey', b'atalh', b'funimation', b'stipulation', b'betting', b'separators', b'pommel', b'sig',
step: 10000, loss: 138.822708
step: 20000, loss: 59.304836
step: 30000, loss: 46.534019
step: 40000, loss: 28.697681
step: 50000, loss: 36.321007
step: 60000, loss: 46.113037
step: 70000, loss: 15.083533
step